# Superconductivity in VCA on a 4-site cluster

- Construct a Hubbard model based on a $2\times2$ cluster where several types of superconductivity are defined:

    - On-site, $s$-wave pairing :
    $$ \hat\Delta = \sum_\mathbf{r} ( c_{\mathbf{r}\uparrow} c_{\mathbf{r}\downarrow} + \text{H.c})$$

    - extended $s$-wave pairing : 
    $$ \hat\Delta = \sum_\mathbf{r} \big[ (c_{\mathbf{r}\uparrow} c_{\mathbf{r}+\hat{x}\downarrow} - c_{\mathbf{r}\downarrow} c_{\mathbf{r}+\hat{x}\uparrow}) + (c_{\mathbf{r}\uparrow} c_{\mathbf{r}+\hat{y}\downarrow} - c_{\mathbf{r}\downarrow} c_{\mathbf{r}+\hat{y}\uparrow}) + \text{H.c}\big] $$

    - singlet, $d_{x^2-y^2}$-wave pairing : 
    $$ \hat\Delta = \sum_\mathbf{r} \big[ (c_{\mathbf{r}\uparrow} c_{\mathbf{r}+\hat{x}\downarrow} - c_{\mathbf{r}\downarrow} c_{\mathbf{r}+\hat{x}\uparrow}) - (c_{\mathbf{r}\uparrow} c_{\mathbf{r}+\hat{y}\downarrow} - c_{\mathbf{r}\downarrow} c_{\mathbf{r}+\hat{y}\uparrow}) + \text{H.c}\big] $$

    - singlet, $d_{xy}$-wave pairing : 
    $$ \hat\Delta = \sum_\mathbf{r} \big[ (c_{\mathbf{r}\uparrow} c_{\mathbf{r}+\hat{x}+\hat{y}\downarrow} - c_{\mathbf{r}\downarrow} c_{\mathbf{r}+\hat{x}+\hat{y}\uparrow}) - (c_{\mathbf{r}\uparrow} c_{\mathbf{r}+\hat{y}-\hat{x}\downarrow} - c_{\mathbf{r}\downarrow} c_{\mathbf{r}+\hat{y}-\hat{x}\uparrow}) + \text{H.c}\big] $$

    (notation : $\hat{x}$ and $\hat{y}$ are the unit lattice vectors in the $x$ and $y$ directions, respectively.)
    
    
- Show how the Potthoff functional $\Omega$ depends on the superconducting Weiss field for each of the cases defined above.

- In the case of $d_{x^2-y^2}$-wave pairing and $U=8$, obtain the VCA solution for a hole-doped system as a function of chemical potential (or doping) for the range of doping where the solution is nontrivial. Plot the superconducting order parameter as a function of doping $x=1-n$, ($n$ begin the electron density).



In [ ]:
from pyqcm import *
from pyqcm.draw_operator import *
from pyqcm.vca import *
from pyqcm.loop import controlled_loop

import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
# Defining a simple 2x2 cluster and inserting into a square-tiled lattice
new_cluster_model("clus", 4, 0)
add_cluster("clus", [0,0,0], [[0,0,0], [1,0,0], [0,1,0], [1,1,0]])
lattice_model("2D_2x2", [[2,0,0],[0,2,0]])

In [ ]:
# Adding the typical Hubbard style operators
interaction_operator("U")
hopping_operator("t", [1,0,0], -1)
hopping_operator("t", [0,1,0], -1)

### It is now time to define a few operators representing superconductivity with singlet pairing

In [ ]:
# On-site s-wave
anomalous_operator("S", [0,0,0], 1)

# Extended s-wave
anomalous_operator("exS", [1,0,0], 1)
anomalous_operator("exS", [0,1,0], 1)

# d_{x^2-y^2}-wave
anomalous_operator("Dx2-y2", [1,0,0], 1)
anomalous_operator("Dx2-y2", [0,1,0], -1)

# d_{xy}-wave, similar as previous wave but defined with "diagonal" links
anomalous_operator("Dxy", [1,1,0], 1)
anomalous_operator("Dxy", [-1,1,0], -1)

In [ ]:
# Defining the appropriate sector of Hilbert space
set_target_sectors(["R0:S0"])

# The first three parameters are used for the graph of the Potthoff functionnal
set_parameters("""
    t=1
    U=8
    mu=1.2
    S_1=1e-9
    exS_1=1e-9
    Dx2-y2_1=1e-9
    Dxy_1=1e-9
""")

In [ ]:
# Generating a grid for the Potthoff functionnals
grid_size = 50
Delta_grid = np.linspace(0.0, 0.3, grid_size)

In [ ]:
# The different types of superconductivity are explored by plotting the Potthoff functionnal for each variety
######################### - On-site s-wave - #########################
set_parameter("S_1", 1)
set_parameter("exS_1", 1e-9)
set_parameter("Dx2-y2_1", 1e-9)
set_parameter("Dxy_1", 1e-9)
new_model_instance()

plot_sef("S_1", Delta_grid)

######################### - Extended s-wave - #########################
set_parameter("S_1", 1e-9)
set_parameter("exS_1", 1)
set_parameter("Dx2-y2_1", 1e-9)
set_parameter("Dxy_1", 1e-9)
new_model_instance()

plot_sef("exS_1", Delta_grid)

######################### - d_{x^2 - y^2}-wave - #########################

set_parameter("S_1", 1e-9)
set_parameter("exS_1", 1e-9)
set_parameter("Dx2-y2_1", 1)
set_parameter("Dxy_1", 1e-9)
new_model_instance()

plot_sef("Dx2-y2_1", Delta_grid)

######################### - d_{xy}-wave - #########################

set_parameter("S_1", 1e-9)
set_parameter("exS_1", 1e-9)
set_parameter("Dx2-y2_1", 1e-9)
set_parameter("Dxy_1", 1)
new_model_instance()

plot_sef("Dxy_1", Delta_grid)

In [ ]:
# Loading the sef.tsv file into an array
raw_data = np.genfromtxt("./sef.tsv", delimiter="\t", usecols=[1])
data = raw_data[-4*grid_size-4:] # reading the last 4 sef simulation blocks accounting for the "omega" header

# Loading each superconducting sef into an array
omega_1 = data[1:grid_size+1]
omega_2 = data[grid_size+2:2*grid_size+2]
omega_3 = data[2*grid_size+3:3*grid_size+3]
omega_4 = data[3*grid_size+4:4*grid_size+4]

In [ ]:
# Plotting all Potthoff functionnals in the same graph
fig, ax = plt.subplots()

ax.plot(Delta_grid, omega_1, label="$s$-wave")
ax.plot(Delta_grid, omega_2, label="extended $s$-wave")
ax.plot(Delta_grid, omega_3, label="$d_{x^2 - y^2}$")
ax.plot(Delta_grid, omega_4, label="$d_{xy}$")

ax.set_aspect(5)
ax.set_xlim((0.0, 0.3))

ax.legend()
ax.set_xlabel("$\Delta$")
ax.set_ylabel("$\Omega$")

fig.show()

### Interpretation of the Potthoff functionnals

As seen in the above plot, the only functionnal that admits a minimum is the one related to $d_{x^2 - y^2}$ superconduction. This implies that it's also the "appropriate" form of superconduction for the model we have defined.

In [ ]:
# Loop_parameters
mu_start = 1.2
mu_stop = 2.8
mu_step = 0.05

In [ ]:
# Setting parameters for a VCA around d_{x^2-y^2} superconduction
set_parameter("S_1", 1e-9)
set_parameter("exS_1", 1e-9)
set_parameter("Dx2-y2_1", 0.15) # Starting parameter for the VCA
set_parameter("Dxy_1", 1e-9)
new_model_instance()

# Defining a fucntion that runs the appropriate VCA every time func() is called in controlled_loop()
def run_vca():
    vca(names=["Dx2-y2_1"], steps=[0.01], accur=[2e-3], max=[100], max_iter=60)

controlled_loop(func=run_vca, varia=["Dx2-y2_1"], loop_param="mu", loop_range=(mu_start, mu_stop, mu_step))

In [ ]:
# Creating a grid for the plot
mu_grid = np.arange(mu_start, mu_stop, mu_step)

# Loading the vca.tsv file into an array
raw_vca_data = np.genfromtxt("./vca.tsv", delimiter="\t", usecols=[3, 10, 12])
vca_data = raw_vca_data[-len(mu_grid):] # loading the appropriate VCA simulations (last N corresponding to controlled_loop)


In [ ]:
fig, ax = plt.subplots()

ax.plot(1-vca_data[:,1], vca_data[:,0], 'o', markersize=2, label="$\Delta$")
ax.plot(1-vca_data[:,1], vca_data[:,2], 'o', markersize=2, label="$\langle\Delta\\rangle$")

ax.set_xlabel("$1-n$")
ax.set_ylabel("$\langle\hat\Delta\rangle$")

ax.legend()

fig.show()


### Conclusion

We can see that the most superconduction is observed for a hole density of $\approx0.095$